In [16]:
from pyspark.sql import SparkSession
from delta import *
from termcolor import cprint

# builder = SparkSession.builder.appName("add_duplicates") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark = SparkSession.builder.appName('duplicate_rows').getOrCreate()

25/08/08 18:42:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [17]:
from pyspark.sql.functions import expr

df = spark.createDataFrame([(1,2,1), (2,9,1), (3,8,2), (4,1,1), (5,3,3)], ["A", "B", "n"])
df.show()
new_df = df.withColumn('n', expr('explode(array_repeat(n,int(n)))'))
new_df.show()

+---+---+---+
|  A|  B|  n|
+---+---+---+
|  1|  2|  1|
|  2|  9|  1|
|  3|  8|  2|
|  4|  1|  1|
|  5|  3|  3|
+---+---+---+

+---+---+---+
|  A|  B|  n|
+---+---+---+
|  1|  2|  1|
|  2|  9|  1|
|  3|  8|  2|
|  3|  8|  2|
|  4|  1|  1|
|  5|  3|  3|
|  5|  3|  3|
|  5|  3|  3|
+---+---+---+



### Add random duplicates of rows from the same dataframe

`sample(replacement, fraction)` - Returns a sampled subset of df,  
 * replacement ==True means duplicate values exist, while false means there are no duplicates 
 * fraction of rows to generate (range[0.0, 1,0])
 
then union duplicates with original df

In [18]:
import random

# randomly add any number or rows from 1 to 3, around the middle of df
with_dup = df.union(df.sample(False, 0.5).limit(random.randint(1,3)))

with_dup.show()

+---+---+---+
|  A|  B|  n|
+---+---+---+
|  1|  2|  1|
|  2|  9|  1|
|  3|  8|  2|
|  4|  1|  1|
|  5|  3|  3|
|  1|  2|  1|
|  2|  9|  1|
+---+---+---+



In [19]:
with_dup = df.union(df.sample(False, 0.1).limit(random.randint(1,3)))

with_dup.show()

+---+---+---+
|  A|  B|  n|
+---+---+---+
|  1|  2|  1|
|  2|  9|  1|
|  3|  8|  2|
|  4|  1|  1|
|  5|  3|  3|
+---+---+---+



In [20]:
with_dup = df.union(df.sample(True, 0.5).limit(random.randint(1,3)))

with_dup.show()

+---+---+---+
|  A|  B|  n|
+---+---+---+
|  1|  2|  1|
|  2|  9|  1|
|  3|  8|  2|
|  4|  1|  1|
|  5|  3|  3|
|  1|  2|  1|
+---+---+---+

